<a href="https://www.kaggle.com/mickaelnarboni/clients-segmentation-rfm-maintenance?scriptVersionId=88947363" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Table of Contents

* [Dataframes frequency](#dataframes-frequency)
* [Clusters stability](#clusters-stability)
* [Adjusted Rand Index](#ari)
* [Final thoughts](#final-thoughts)


Import the relevant library for the notebook.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore") # ignore the warnings about file size
import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib inline
import seaborn as sns
from time import process_time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import math
from math import *
from sklearn.metrics import adjusted_rand_score
import plotly.io as pio
pio.renderers.default = 'iframe'
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import MiniBatchKMeans
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from yellowbrick.contrib.scatter import ScatterVisualizer
import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.express as px


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/olist-clients-segmentation/df_segmentation.csv
/kaggle/input/olist-clients-segmentation/database_p4.csv
/kaggle/input/olist-clients-segmentation/geolocation_p4.csv


Import our dataframe. 

In [2]:
df = pd.read_csv('../input/olist-clients-segmentation/df_segmentation.csv',sep='\t', index_col=[0], low_memory=False)
df

,CustomerUniqueID,MaxPurchaseDate,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
0,861eff4711a542e4b93843c6dd7febb0,2017-05-16 15:05:35,474,2,1,0,124.99,0,1.078731,-0.574149,0.163933,0,Bronze
1,290c77bc529b7ac935b93aa66c333dc3,2018-01-12 20:48:24,233,0,1,0,289.00,0,0.283691,-0.574149,0.981946,0,Bronze
2,060e732b5b29e8181a18229c7b0b2b5e,2018-05-19 16:07:45,106,3,1,0,139.94,0,-0.598033,-0.574149,0.274193,0,Bronze
3,259dac757896d24d7702b9acbbff3f3c,2018-03-13 16:06:38,173,0,1,0,149.94,0,-0.049639,-0.574149,0.341553,0,Bronze
4,345ecd01c38d18a9036ed96c73b8d066,2018-07-29 09:51:30,35,3,1,0,230.00,0,-1.838550,-0.574149,0.759096,3,Gold
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115604,1a29b476fee25c95fbafc67c5ac95cf8,2018-04-07 15:48:17,148,0,1,0,74.90,0,-0.224371,-0.574149,-0.335818,1,Diamond
115605,d52a67c98be1cf6a5c84435bd38d095d,2018-04-04 08:20:22,152,0,1,0,114.90,0,-0.194516,-0.574149,0.081788,0,Bronze
115606,e9f50caf99f032f0bf3c55141f019d99,2018-04-08 20:11:50,147,0,1,0,37.00,0,-0.231961,-0.574149,-1.024074,1,Diamond
115607,73c2643a0a458b49f58cea58833b192e,2017-11-03 21:08:33,303,1,1,0,689.00,3,0.577779,-0.574149,1.829843,0,Bronze


Return the shape of the dataframe. 

In [3]:
df.shape

(115609, 13)

We order the entries by Recency so it'll be easier for us to create regular interval of recency to split our customers.
We notice that our latest customer order has been fulfilled 728 days ago (almost 2 years) before downloading the Olist data.

In [4]:
df.sort_values(by=['MaxPurchaseDate'], ascending=True, inplace=True)
df

,CustomerUniqueID,MaxPurchaseDate,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
99530,b7d76e111c89f7ebf14761390f0f7d17,2016-09-04 21:15:19,728,2,2,0,72.89,0,1.559105,0.625897,-0.362365,1,Diamond
99531,b7d76e111c89f7ebf14761390f0f7d17,2016-09-04 21:15:19,728,2,2,0,72.89,0,1.559105,0.625897,-0.362365,1,Diamond
83867,4854e9b3feff728c13ee5fc7d1547e92,2016-09-05 00:15:34,728,2,1,0,59.50,0,1.559105,-0.574149,-0.560453,1,Diamond
105686,0eb1ee9dba87f5b36b4613a65074337c,2016-10-02 22:07:52,700,2,1,0,100.00,0,1.515197,-0.574149,-0.053760,0,Bronze
66386,2f64e403852e6893ae37485d5fcacdaf,2016-10-03 16:56:50,699,2,1,0,21.90,0,1.513596,-0.574149,-1.535879,1,Diamond
...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,7febafa06d9d8f232a900a2937f04338,2018-08-29 14:18:28,4,3,1,0,45.90,0,-4.266824,-0.574149,-0.813717,3,Gold
60274,b701bebbdf478f5500348f03aff62121,2018-08-29 14:52:00,4,3,1,0,24.90,0,-4.266824,-0.574149,-1.410589,3,Gold
80199,24ac2b4327e25baf39f2119e4228976a,2018-08-29 15:00:37,4,3,1,0,84.99,0,-4.266824,-0.574149,-0.212481,3,Gold
49668,ff22e30958c13ffe219db7d711e8f564,2018-09-03 09:06:57,0,3,2,0,290.00,0,-5.818792,0.625897,0.985317,3,Gold


<a id="dataframes-frequency"></a>
## Dataframes frequency

We keep only the month and year of the MaxPurchaseDate column to have a better split of our customers based on the month. 

In [5]:
df['MaxPurchaseDate'] = df['MaxPurchaseDate'].apply(lambda x: x[:7])
df

,CustomerUniqueID,MaxPurchaseDate,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
99530,b7d76e111c89f7ebf14761390f0f7d17,2016-09,728,2,2,0,72.89,0,1.559105,0.625897,-0.362365,1,Diamond
99531,b7d76e111c89f7ebf14761390f0f7d17,2016-09,728,2,2,0,72.89,0,1.559105,0.625897,-0.362365,1,Diamond
83867,4854e9b3feff728c13ee5fc7d1547e92,2016-09,728,2,1,0,59.50,0,1.559105,-0.574149,-0.560453,1,Diamond
105686,0eb1ee9dba87f5b36b4613a65074337c,2016-10,700,2,1,0,100.00,0,1.515197,-0.574149,-0.053760,0,Bronze
66386,2f64e403852e6893ae37485d5fcacdaf,2016-10,699,2,1,0,21.90,0,1.513596,-0.574149,-1.535879,1,Diamond
...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,7febafa06d9d8f232a900a2937f04338,2018-08,4,3,1,0,45.90,0,-4.266824,-0.574149,-0.813717,3,Gold
60274,b701bebbdf478f5500348f03aff62121,2018-08,4,3,1,0,24.90,0,-4.266824,-0.574149,-1.410589,3,Gold
80199,24ac2b4327e25baf39f2119e4228976a,2018-08,4,3,1,0,84.99,0,-4.266824,-0.574149,-0.212481,3,Gold
49668,ff22e30958c13ffe219db7d711e8f564,2018-09,0,3,2,0,290.00,0,-5.818792,0.625897,0.985317,3,Gold


We reset the index.

In [6]:
df.reset_index(inplace=True)
df

,index,CustomerUniqueID,MaxPurchaseDate,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
0,99530,b7d76e111c89f7ebf14761390f0f7d17,2016-09,728,2,2,0,72.89,0,1.559105,0.625897,-0.362365,1,Diamond
1,99531,b7d76e111c89f7ebf14761390f0f7d17,2016-09,728,2,2,0,72.89,0,1.559105,0.625897,-0.362365,1,Diamond
2,83867,4854e9b3feff728c13ee5fc7d1547e92,2016-09,728,2,1,0,59.50,0,1.559105,-0.574149,-0.560453,1,Diamond
3,105686,0eb1ee9dba87f5b36b4613a65074337c,2016-10,700,2,1,0,100.00,0,1.515197,-0.574149,-0.053760,0,Bronze
4,66386,2f64e403852e6893ae37485d5fcacdaf,2016-10,699,2,1,0,21.90,0,1.513596,-0.574149,-1.535879,1,Diamond
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115604,624,7febafa06d9d8f232a900a2937f04338,2018-08,4,3,1,0,45.90,0,-4.266824,-0.574149,-0.813717,3,Gold
115605,60274,b701bebbdf478f5500348f03aff62121,2018-08,4,3,1,0,24.90,0,-4.266824,-0.574149,-1.410589,3,Gold
115606,80199,24ac2b4327e25baf39f2119e4228976a,2018-08,4,3,1,0,84.99,0,-4.266824,-0.574149,-0.212481,3,Gold
115607,49668,ff22e30958c13ffe219db7d711e8f564,2018-09,0,3,2,0,290.00,0,-5.818792,0.625897,0.985317,3,Gold


We split the dataframe into business quarter (= 3 months period of time) based from the latest to the newest recency of clients to know until what point the model will be obsolete and need a maintenance work by Olist.

We start on January 2017 (Q1 2017) and end on September 2018 (Q4 2018).

In [7]:
lim = df[df['MaxPurchaseDate'] == '2017-01'].index[-1]
lim_0 = df[df['MaxPurchaseDate'] == '2017-03'].index[-1]
lim_1 = df[df['MaxPurchaseDate'] == '2017-06'].index[-1]
lim_2 = df[df['MaxPurchaseDate'] == '2017-09'].index[-1]
lim_3 = df[df['MaxPurchaseDate'] == '2017-12'].index[-1]
lim_4 = df[df['MaxPurchaseDate'] == '2018-03'].index[-1]
lim_5 = df[df['MaxPurchaseDate'] == '2018-06'].index[-1]
lim_6 = df[df['MaxPurchaseDate'] == '2018-09'].index[-1]

Create B0 subdataframe.

In [8]:
B0 = df[(df['MaxPurchaseDate'].index <= lim_0) & (df['MaxPurchaseDate'].index >= lim)]
B0

,index,CustomerUniqueID,MaxPurchaseDate,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
1330,18118,e264ef071f4c0fb4a0f3e9139b283dee,2017-01,579,2,1,0,239.99,0,1.302739,-0.574149,0.800591,0,Bronze
1331,96381,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1332,96382,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1333,66517,4fd565778fe054e28d5aba1088700075,2017-02,579,2,1,0,48.00,0,1.302739,-0.574149,-0.770058,1,Diamond
1334,97215,ef057d5e1e98631079064c5b892de94c,2017-02,578,2,1,0,129.99,0,1.300804,-0.574149,0.202213,0,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6287,1249,589ee29a3decf9bd5cd8492da02efb2e,2017-03,520,2,1,0,109.70,0,1.182421,-0.574149,0.036591,0,Bronze
6288,16863,fc1c186e0436f5b790ab26eea95aafff,2017-03,520,2,1,0,48.00,0,1.182421,-0.574149,-0.770058,1,Diamond
6289,17466,f389ca60b51c77eba0045a6789014b42,2017-03,520,2,1,0,116.00,0,1.182421,-0.574149,0.091087,0,Bronze
6290,29577,88096271a97e2192b1dbd67c4b06d394,2017-03,520,2,1,0,69.90,0,1.182421,-0.574149,-0.403242,1,Diamond


Create B1 subdataframe.

In [9]:
B1 = df[(df['MaxPurchaseDate'].index < lim_1) & (df['MaxPurchaseDate'].index >= lim)]
B1

,index,CustomerUniqueID,MaxPurchaseDate,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
1330,18118,e264ef071f4c0fb4a0f3e9139b283dee,2017-01,579,2,1,0,239.99,0,1.302739,-0.574149,0.800591,0,Bronze
1331,96381,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1332,96382,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1333,66517,4fd565778fe054e28d5aba1088700075,2017-02,579,2,1,0,48.00,0,1.302739,-0.574149,-0.770058,1,Diamond
1334,97215,ef057d5e1e98631079064c5b892de94c,2017-02,578,2,1,0,129.99,0,1.300804,-0.574149,0.202213,0,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16770,71770,cfd6ea17db8bbe26b8f20a13dcd550af,2017-06,429,2,1,0,156.99,0,0.967059,-0.574149,0.386394,0,Bronze
16771,54425,9459c6a0c2fac25ee571ab4d9f18757b,2017-06,429,2,1,0,55.13,0,0.967059,-0.574149,-0.634899,1,Diamond
16772,111631,bb1770ae9e096a25630b2404ed9bcbc4,2017-06,429,2,2,0,45.80,0,0.967059,0.625897,-0.815845,1,Diamond
16773,111630,bb1770ae9e096a25630b2404ed9bcbc4,2017-06,429,2,2,0,45.80,0,0.967059,0.625897,-0.815845,1,Diamond


Create B2 subdataframe.

In [10]:
B2 = df[(df['MaxPurchaseDate'].index < lim_2) & (df['MaxPurchaseDate'].index >= lim)]
B2

,index,CustomerUniqueID,MaxPurchaseDate,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
1330,18118,e264ef071f4c0fb4a0f3e9139b283dee,2017-01,579,2,1,0,239.99,0,1.302739,-0.574149,0.800591,0,Bronze
1331,96381,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1332,96382,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1333,66517,4fd565778fe054e28d5aba1088700075,2017-02,579,2,1,0,48.00,0,1.302739,-0.574149,-0.770058,1,Diamond
1334,97215,ef057d5e1e98631079064c5b892de94c,2017-02,578,2,1,0,129.99,0,1.300804,-0.574149,0.202213,0,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31311,94058,331a90f871396eca3dbbbaafc0f7fb39,2017-09,337,1,1,0,44.90,0,0.696839,-0.574149,-0.835214,1,Diamond
31312,112791,19090a836e2392cd957ed43d61804ba3,2017-09,337,1,1,0,39.00,0,0.696839,-0.574149,-0.972698,1,Diamond
31313,3987,f1e036dfac59643f7bdd306447f92402,2017-09,337,1,1,0,109.90,0,0.696839,-0.574149,0.038368,0,Bronze
31314,114024,3581031fd294025f4c16cd754bc4c90e,2017-09,337,1,1,0,69.90,0,0.696839,-0.574149,-0.403242,1,Diamond


Create B3 subdataframe.

In [11]:
B3 = df[(df['MaxPurchaseDate'].index < lim_3) & (df['MaxPurchaseDate'].index >= lim)]
B3

,index,CustomerUniqueID,MaxPurchaseDate,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
1330,18118,e264ef071f4c0fb4a0f3e9139b283dee,2017-01,579,2,1,0,239.99,0,1.302739,-0.574149,0.800591,0,Bronze
1331,96381,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1332,96382,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1333,66517,4fd565778fe054e28d5aba1088700075,2017-02,579,2,1,0,48.00,0,1.302739,-0.574149,-0.770058,1,Diamond
1334,97215,ef057d5e1e98631079064c5b892de94c,2017-02,578,2,1,0,129.99,0,1.300804,-0.574149,0.202213,0,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51933,73872,e6797b7d4c35624ef13982ab27d3255b,2017-12,245,0,1,0,49.99,0,0.339912,-0.574149,-0.730414,1,Diamond
51934,111387,addd42012ed67639f9e0d26155b7c2e1,2017-12,245,0,1,0,120.90,0,0.339912,-0.574149,0.131465,0,Bronze
51935,51819,b890928f02ba9a0dfbc6ec8009bee0fd,2017-12,245,0,1,0,219.90,0,0.339912,-0.574149,0.715271,0,Bronze
51936,18913,5d11f19985edbb3c068e805434a80f87,2017-12,245,0,2,0,178.00,0,0.339912,0.625897,0.508971,0,Bronze


Create B4 subdataframe.

In [12]:
B4 = df[(df['MaxPurchaseDate'].index < lim_4) & (df['MaxPurchaseDate'].index >= lim)]
B4

,index,CustomerUniqueID,MaxPurchaseDate,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
1330,18118,e264ef071f4c0fb4a0f3e9139b283dee,2017-01,579,2,1,0,239.99,0,1.302739,-0.574149,0.800591,0,Bronze
1331,96381,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1332,96382,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1333,66517,4fd565778fe054e28d5aba1088700075,2017-02,579,2,1,0,48.00,0,1.302739,-0.574149,-0.770058,1,Diamond
1334,97215,ef057d5e1e98631079064c5b892de94c,2017-02,578,2,1,0,129.99,0,1.300804,-0.574149,0.202213,0,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76471,33831,fed18cac60842f3e18f69218ee25a64a,2018-03,155,0,1,0,190.00,0,-0.172636,-0.574149,0.572641,0,Bronze
76472,39570,a3e9d9b99ef084bfc5dc9392d54cb434,2018-03,155,0,1,0,84.90,0,-0.172636,-0.574149,-0.213515,1,Diamond
76473,15943,4e120f53d49d0be0732e809a4a1f1d02,2018-03,155,0,1,0,45.00,0,-0.172636,-0.574149,-0.833042,1,Diamond
76474,109827,d92d360562e974e2cfeee5589472bcf3,2018-03,155,0,1,0,50.40,0,-0.172636,-0.574149,-0.722442,1,Diamond


Create B5 subdataframe.

In [13]:
B5 = df[(df['MaxPurchaseDate'].index < lim_5) & (df['MaxPurchaseDate'].index >= lim)]
B5

,index,CustomerUniqueID,MaxPurchaseDate,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
1330,18118,e264ef071f4c0fb4a0f3e9139b283dee,2017-01,579,2,1,0,239.99,0,1.302739,-0.574149,0.800591,0,Bronze
1331,96381,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1332,96382,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1333,66517,4fd565778fe054e28d5aba1088700075,2017-02,579,2,1,0,48.00,0,1.302739,-0.574149,-0.770058,1,Diamond
1334,97215,ef057d5e1e98631079064c5b892de94c,2017-02,578,2,1,0,129.99,0,1.300804,-0.574149,0.202213,0,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100529,18362,647492daf4b0d652f624abb53f5c7f4f,2018-06,64,3,1,0,25.00,0,-1.162888,-0.574149,-1.406678,3,Gold
100530,63734,637ea98cd161c62ef22d4c9c9a9de6b6,2018-06,64,3,1,0,349.90,0,-1.162888,-0.574149,1.168563,3,Gold
100531,65955,416fe0cd730f41a424697093f013f217,2018-06,64,3,1,0,9.99,0,-1.162888,-0.574149,-2.301884,1,Diamond
100532,58319,549cd16e81ef53f1d64ef7f60a691eea,2018-06,64,3,1,0,130.00,0,-1.162888,-0.574149,0.202288,3,Gold


Create B6 subdataframe.

In [14]:
B6 = df[(df['MaxPurchaseDate'].index < lim_6) & (df['MaxPurchaseDate'].index >= lim)]
B6

,index,CustomerUniqueID,MaxPurchaseDate,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
1330,18118,e264ef071f4c0fb4a0f3e9139b283dee,2017-01,579,2,1,0,239.99,0,1.302739,-0.574149,0.800591,0,Bronze
1331,96381,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1332,96382,13aec51202205c662b24202760e93a92,2017-02,579,2,2,0,64.90,0,1.302739,0.625897,-0.475674,1,Diamond
1333,66517,4fd565778fe054e28d5aba1088700075,2017-02,579,2,1,0,48.00,0,1.302739,-0.574149,-0.770058,1,Diamond
1334,97215,ef057d5e1e98631079064c5b892de94c,2017-02,578,2,1,0,129.99,0,1.300804,-0.574149,0.202213,0,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115603,99432,5c58de6fb80e93396e2f35642666b693,2018-08,4,3,2,0,418.00,0,-4.266824,0.625897,1.342116,3,Gold
115604,624,7febafa06d9d8f232a900a2937f04338,2018-08,4,3,1,0,45.90,0,-4.266824,-0.574149,-0.813717,3,Gold
115605,60274,b701bebbdf478f5500348f03aff62121,2018-08,4,3,1,0,24.90,0,-4.266824,-0.574149,-1.410589,3,Gold
115606,80199,24ac2b4327e25baf39f2119e4228976a,2018-08,4,3,1,0,84.99,0,-4.266824,-0.574149,-0.212481,3,Gold


We create a dataframe for each subdataframe that will contain our three RFM variables after transformation. 

In [15]:
rfm_B0 = B0[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B1 = B1[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B2 = B2[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B3 = B3[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B4 = B4[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B5 = B5[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B6 = B6[['Recency_standard','Frequency_standard','Monetary_value_standard']]

<a id="clusters-stability"></a>
## Clusters stability

Create a KMeans method with the same hyperparameters tested in the modeling notebook.

In [16]:
# Use the hyperparameters from our model K=4 using KMeans clustering method
kmeans_cluster = KMeans(n_clusters=4, n_init=10, init='k-means++', max_iter=100)

Create a model M(i) to fit each subdataframe B(i).

In [17]:
M0 = kmeans_cluster.fit(rfm_B0)
M1 = kmeans_cluster.fit(rfm_B1)
M2 = kmeans_cluster.fit(rfm_B2)
M3 = kmeans_cluster.fit(rfm_B3)
M4 = kmeans_cluster.fit(rfm_B4)
M5 = kmeans_cluster.fit(rfm_B5)
M6 = kmeans_cluster.fit(rfm_B6)

Create the arrays of labels of our model M0 to predict each subdataframe B(i).

In [18]:
label_00 = M0.predict(rfm_B0)
label_01 = M0.predict(rfm_B1)
label_02 = M0.predict(rfm_B2)
label_03 = M0.predict(rfm_B3)
label_04 = M0.predict(rfm_B4)
label_05 = M0.predict(rfm_B5)
label_06 = M0.predict(rfm_B6)

Create the arrays of labels of our model M(i) to fit and predict the subdataframes B(i).

In [19]:
label_11 = M1.fit_predict(rfm_B1)
label_22 = M2.fit_predict(rfm_B2)
label_33 = M3.fit_predict(rfm_B3)
label_44 = M4.fit_predict(rfm_B4)
label_55 = M5.fit_predict(rfm_B5)
label_66 = M6.fit_predict(rfm_B6)

Now, we want to compare our model M0 that predict B(i) to each model M(i) that fit and predict M(i).

In [20]:
ari0 = adjusted_rand_score(label_00, label_00)
ari1 = adjusted_rand_score(label_01, label_11)
ari2 = adjusted_rand_score(label_02, label_22)
ari3 = adjusted_rand_score(label_03, label_33)
ari4 = adjusted_rand_score(label_04, label_44)
ari5 = adjusted_rand_score(label_05, label_55)
ari6 = adjusted_rand_score(label_06, label_66)

<a id="ari"></a>
## Adjusted Rand Index

The ARI allows us to score our model stability to see if the predicted clustering are sorted in the same clusters as our model. The closer the score is from 1, the better.
We do that on the three variables we've been building in our model notebook: Recency, Frequency, Monetary Value.
We apply the models to each new dataframe we previously created to track the evolution of the stability along time.

We create a table that contains all our ARI scores. 

In [21]:
d = {'ARI'}
list1 = [ari0, ari1, ari2, ari3, ari4, ari5]
ari_table = pd.DataFrame(data = list1, columns=d)
ari_table

,ARI
0,1.000000
1,0.535166
2,0.607128
3,0.582817
4,0.530014
5,0.522777


Plot a curve of the ARI on each subdataframe.
We get the ARI for each new 10,000 orders based on recency from the latest to the newest.

In [22]:
pd.options.plotting.backend = "plotly"
fig = ari_table.plot(title = "RFM Predictions per quarter for clients segmentation with KMeans using ARI", labels=dict(index="Quarter", value="Precision Score", variable=""))
fig.show()

<a id="final-thoughts"></a>
## Final thoughts

We notice a big drop at Q2 2017 (3 months period of time) and another drop 1 year later at Q1 2018.

We recommend Olist to update the clustering model **at the end of each quarter** for an accurate model. 